# Speed Up Python Code

👉🏻 **Example**: _Matrix multiplication_

```python
import random


class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])
    
    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255) for j in range(n_cols)])
        return M
    
    @property
    def shape(self):
        return ((0, 0) if not self else (len(self), len(self[0])))


def matrix_product(X, Y):
    """Computes the matrix product of X and Y.

    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6])
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # TODO: check the shapes
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z
```

## Measuring execution time

In [1]:
import timeit

In [2]:
setup = """from fast_python import Matrix, matrix_product
shape = 64, 64
X = Matrix.random(shape)
Y = Matrix.random(shape)
"""

In [3]:
timeit.timeit("matrix_product(X, Y)", setup, number=10)

0.884664481999998

⚠️ _`timeit` disables GC._

### IPython

In [4]:
from fast_python import Matrix, matrix_product

In [5]:
shape = 64, 64

In [6]:
X, Y = Matrix.random(shape), Matrix.random(shape)

In [7]:
%timeit matrix_product(X, Y)

86.3 ms ± 324 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit -n100 matrix_product(X, Y)

89 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Looking for performance problems

🤔 What wrong with this code?

```python
def matrix_product(X, Y):
    """Computes the matrix product of X and Y.

    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6])
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # TODO: check the shapes
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z
```

### The `cProfile` module

In [7]:
import cProfile

In [8]:
source = open("fast_python.py").read()

In [9]:
cProfile.run(source, sort="tottime")

         41381 function calls in 1.446 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    1.426    0.089    1.426    0.089 <string>:22(matrix_product)
     8192    0.006    0.000    0.013    0.000 random.py:174(randrange)
     8192    0.005    0.000    0.007    0.000 random.py:224(_randbelow)
     8192    0.003    0.000    0.016    0.000 random.py:218(randint)
      128    0.002    0.000    0.018    0.000 <string>:14(<listcomp>)
     8205    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.001    0.001    1.445    1.445 <string>:42(bench)
     8192    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
        1    0.000    0.000    1.446    1.446 {built-in method builtins.exec}
       16    0.000    0.000    0.000    0.000 <string>:8(<listcomp>)
        2    0.000    0.000    0.018    0.009 <string>:10(random)
        1    0.000    0.000    1.4

Profiler doesn't help much here.

### The `line_profiler` module

In [24]:
!pip3 install line_profiler

     |████████████████████████████████| 45 kB 1.0 MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for line-profiler: filename=line_profiler-3.0.2-cp37-cp37m-macosx_10_15_x86_64.whl size=55572 sha256=e44add9e8193ce9a79ebbdf44cfae5ea732665c9fd7d2dd25c47e0a5d66c8468
  Stored in directory: /Users/akrisanov/Library/Caches/pip/wheels/20/28/5c/69ef18a2dc224b4230d6b88e9c0721ec7d3bb425b71d3248ce
Successfully built line-profiler


In [1]:
%load_ext line_profiler

In [2]:
from fast_python import matrix_product, bench

In [3]:
%lprun -f matrix_product bench()

```shell
Timer unit: 1e-06 s

Total time: 4.25494 s
File: /Users/akrisanov/github/python_notebook/fast_python.py
Function: matrix_product at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    22                                           def matrix_product(X, Y):
    23                                               """Computes the matrix product of X and Y.
    24
    25                                               >>> X = Matrix([[1], [2], [3]])
    26                                               >>> Y = Matrix([[4, 5, 6])
    27                                               [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    28                                               >> matrix_product(Y, X)
    29                                               [[32]]
    30                                               """
    31        16         71.0      4.4      0.0      n_xrows, n_xcols = X.shape
    32        16         44.0      2.8      0.0      n_yrows, n_ycols = Y.shape
    33                                               # TODO: check the shapes
    34        16        431.0     26.9      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
    35      1040        352.0      0.3      0.0      for i in range(n_xrows):
    36     66560      20002.0      0.3      0.5          for j in range(n_xcols):
    37   4259840    1320290.0      0.3     31.0              for k in range(n_ycols):
    38   4194304    2913738.0      0.7     68.5                  Z[i][k] += X[i][j] * Y[j][k]
    39        16          8.0      0.5      0.0      return Z
```

## Optimizing list access

💡 `list.__getitem__` is not a free operation. We can try to remember values `X[i]` and `Z[i]` and reduce access by index in the loops.

In [5]:
# version 2
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] += acc
    return Z

## Using generator expressions

In [6]:
# version 3
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            # sum is builtins function written in C
            Z[i][k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

## Transposing matrix

In [10]:
# version 4
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

## Other Options

- NumPy
- Ahead-of-time compilation: Python C-API, Cython
- Just-in-time compilation: PyPy, Numba, HOPE

## Resources

* [timeit — Measure execution time of small code snippets](https://docs.python.org/3/library/timeit.html)
* [The Python Profilers](https://docs.python.org/3/library/profile.html)
* [pycallgraph](https://github.com/gak/pycallgraph/)
* [Pyflame: A Ptracing Profiler For Python](https://github.com/uber/pyflame)